In [1]:
# imports
import pandas as p                              # it's pandas
from sklearn.preprocessing import MinMaxScaler  # for features scaling
import Constants as c                           # constants
import Functions as f                           # plotting functions
import numpy as np
import pickle
import folium
from folium.plugins import HeatMap
import webbrowser as wb
import os

In [2]:
# Import dataframe from "out" folder
with open('../out/TREAD_out.pkl', 'rb') as file:
    df, routes_list, name_of_route = pickle.load(file)

test_route = routes_list[1]

In [3]:
# some info about the dataframe

test_route.reset_index(inplace=True, drop=True)
if 'index' in test_route.columns:
    print("Column \"Index\" removed")
    # the above instruction sometimes adds a new column called 'index', which is subsequently removed
    test_route.drop(['index'], axis=1, inplace=True)
display(test_route)
#display(df)
# current columns (aka features) of the dataset
print(df.columns)
display(df.info())

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,VesselType,Status,...,Draft,IsClassA,Avg_Speed,EstimatedStatus,IsInWP,IsInSignificantWP,Route,IsInRoute,group,route_id
0,338143427,2019-01-08 00:32:22,20.7358,-156.8890,6.9,116.000000,511.0,KOA,36,99,...,NaN,False,0.53,sailing,WP_0,<NA>,WP_0 - WP_33,True,1,2189
1,338143427,2019-01-08 00:33:52,20.7364,-156.8862,6.3,74.800003,511.0,KOA,36,99,...,NaN,False,6.46,sailing,WP_0,<NA>,WP_0 - WP_33,True,0,2189
2,338143427,2019-01-08 00:36:21,20.7376,-156.8818,6.0,76.300003,511.0,KOA,36,99,...,NaN,False,6.22,sailing,WP_0,<NA>,WP_0 - WP_33,True,0,2189
3,338143427,2019-01-08 00:37:22,20.7381,-156.8800,6.1,74.900002,511.0,KOA,36,99,...,NaN,False,6.23,sailing,WP_0,<NA>,WP_0 - WP_33,True,0,2189
4,338143427,2019-01-08 00:38:54,20.7388,-156.8774,6.1,64.500000,511.0,KOA,36,99,...,NaN,False,5.95,sailing,WP_0,<NA>,WP_0 - WP_33,True,0,2189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,338143427,2019-01-08 03:13:51,20.8544,-156.6858,5.8,28.500000,511.0,KOA,36,99,...,NaN,False,6.01,sailing,WP_33,<NA>,WP_0 - WP_33,True,0,2189
101,338143427,2019-01-08 03:14:53,20.8558,-156.6850,6.0,27.200001,511.0,KOA,36,99,...,NaN,False,5.54,sailing,WP_33,<NA>,WP_0 - WP_33,True,0,2189
102,338143427,2019-01-08 03:16:22,20.8581,-156.6843,5.8,14.400000,511.0,KOA,36,99,...,NaN,False,5.81,sailing,WP_33,<NA>,WP_0 - WP_33,True,0,2189
103,338143427,2019-01-08 03:17:52,20.8606,-156.6840,6.2,13.300000,511.0,KOA,36,99,...,NaN,False,6.05,sailing,WP_33,<NA>,WP_0 - WP_33,True,0,2189


Index(['MMSI', 'BaseDateTime', 'LAT', 'LON', 'SOG', 'COG', 'Heading',
       'VesselName', 'VesselType', 'Status', 'Length', 'Width', 'Draft',
       'IsClassA', 'Avg_Speed', 'EstimatedStatus', 'IsInWP',
       'IsInSignificantWP', 'Route', 'IsInRoute', 'group', 'route_id'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Index: 5415129 entries, 0 to 2371
Data columns (total 22 columns):
 #   Column             Dtype        
---  ------             -----        
 0   MMSI               int32        
 1   BaseDateTime       datetime64[s]
 2   LAT                float64      
 3   LON                float64      
 4   SOG                float64      
 5   COG                float32      
 6   Heading            float32      
 7   VesselName         string       
 8   VesselType         int16        
 9   Status             int8         
 10  Length             int16        
 11  Width              int16        
 12  Draft              float32      
 13  IsClassA           boo

None

#### General Approach
1. Normalization (some features have different scales) (to be decided, probably not)
2. Features selection
3. Algorithm

### Normalization

In [4]:
test_route.isnull().sum()

MMSI                   0
BaseDateTime           0
LAT                    0
LON                    0
SOG                    0
COG                    0
Heading                0
VesselName             0
VesselType             0
Status                 0
Length                 0
Width                  0
Draft                105
IsClassA               0
Avg_Speed              0
EstimatedStatus        0
IsInWP                13
IsInSignificantWP    105
Route                  0
IsInRoute              0
group                  0
route_id               0
dtype: int64

In [5]:
# scaling, to be decided
'''
df = df.sort_values(by=['MMSI','BaseDateTime'])
#selected_cols = ['LAT','LON','SOG','COG','Heading','Length','Width','Avg_Speed']
selected_cols = ['LAT','LON','SOG','COG','Heading','Avg_Speed']

scaler = MinMaxScaler()
df[cols_to_be_scaled] = scaler.fit_transform(df[cols_to_be_scaled])

display(df.head(3))

# create a list of routes from the scaled dataframe df


only_route_points = df[(df['IsInRoute'] == True) & (df['Route'] == name_of_route)]

list_of_routes = [route[selected_cols]   for _, route in only_route_points.groupby('route_id')]


display(only_route_points.head())

display(list_of_routes[0])
'''

"\ndf = df.sort_values(by=['MMSI','BaseDateTime'])\n#selected_cols = ['LAT','LON','SOG','COG','Heading','Length','Width','Avg_Speed']\nselected_cols = ['LAT','LON','SOG','COG','Heading','Avg_Speed']\n\nscaler = MinMaxScaler()\ndf[cols_to_be_scaled] = scaler.fit_transform(df[cols_to_be_scaled])\n\ndisplay(df.head(3))\n\n# create a list of routes from the scaled dataframe df\n\n\nonly_route_points = df[(df['IsInRoute'] == True) & (df['Route'] == name_of_route)]\n\nlist_of_routes = [route[selected_cols]   for _, route in only_route_points.groupby('route_id')]\n\n\ndisplay(only_route_points.head())\n\ndisplay(list_of_routes[0])\n"

### Algorithm

In [6]:
'''
testDF = p.concat(routes_list, ignore_index=True, axis=0)

# create Heatmap using Folium
map = folium.Map(location=[testDF.LAT.mean(), testDF.LON.mean()], zoom_start=12)

chosen_columns = ['LAT', 'LON', 'Avg_Speed', 'COG', 'Heading', 'IsClassA']
heat_data = testDF[['LAT', 'LON']].values.tolist()
heat_data_DF = testDF[chosen_columns]

HeatMap(heat_data, radius=14).add_to(map)

map.save('test.html')
wb.open('file://' + os.path.realpath('test.html'))
'''


"\ntestDF = p.concat(routes_list, ignore_index=True, axis=0)\n\n# create Heatmap using Folium\nmap = folium.Map(location=[testDF.LAT.mean(), testDF.LON.mean()], zoom_start=12)\n\nchosen_columns = ['LAT', 'LON', 'Avg_Speed', 'COG', 'Heading', 'IsClassA']\nheat_data = testDF[['LAT', 'LON']].values.tolist()\nheat_data_DF = testDF[chosen_columns]\n\nHeatMap(heat_data, radius=14).add_to(map)\n\nmap.save('test.html')\nwb.open('file://' + os.path.realpath('test.html'))\n"

In [7]:
#another test

testDF = p.concat(routes_list, ignore_index=True, axis=0)
chosen_columns = ['LAT', 'LON', 'Avg_Speed', 'COG', 'Heading', 'IsClassA']
chosen_data = testDF[chosen_columns]

from sklearn.mixture import GaussianMixture

lowest_bic = np.inf
best_gmm = None
n_clusters_range = range(1,11)


for cluster_num in n_clusters_range:
    gmm = GaussianMixture(n_components= cluster_num, random_state = 0)
    gmm.fit(chosen_data)
    bic = gmm.bic(chosen_data)

    if bic < lowest_bic:
        lowest_bic = bic
        best_gmm = gmm

testDF['cluster'] = best_gmm.predict(chosen_data)

map = folium.Map(location=[testDF.LAT.mean(), testDF.LON.mean()], zoom_start=12)

colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen']

#plot

f.route_clusters_plot(testDF)